# Data Preprocessing
:label:`sec_pandas`

So far, we have been working with synthetic data
that arrived in ready-made tensors.
However, to apply deep learning in the wild
we must extract messy data 
stored in arbitrary formats,
and preprocess it to suit our needs.
Fortunately, the *pandas* [library](https://pandas.pydata.org/) 
can do much of the heavy lifting.
This section, while no substitute 
for a proper *pandas* [tutorial](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html),
will give you a crash course
on some of the most common routines.

## Reading the Dataset

Comma-separated values (CSV) files are ubiquitous 
for the storing of tabular (spreadsheet-like) data.
In them, each line corresponds to one record
and consists of several (comma-separated) fields, e.g.,
"Albert Einstein,March 14 1879,Ulm,Federal polytechnic school,field of gravitational physics".
To demonstrate how to load CSV files with `pandas`, 
we (**create a CSV file below**) `../data/house_tiny.csv`. 
This file represents a dataset of homes,
where each row corresponds to a distinct home
and the columns correspond to the number of rooms (`NumRooms`),
the roof type (`RoofType`), and the price (`Price`).


In [1]:
import os

os.makedirs(os.path.join('..', 'data'), exist_ok=True)
data_file = os.path.join('..', 'data', 'house_tiny.csv')
with open(data_file, 'w') as f:
    f.write('''NumRooms,RoofType,Price
NA,NA,127500
2,NA,106000
4,Slate,178100
NA,NA,140000''')

Now let's import `pandas` and load the dataset with `read_csv`.


In [2]:
import pandas as pd

data = pd.read_csv(data_file)
print(data)

   NumRooms RoofType   Price
0       NaN      NaN  127500
1       2.0      NaN  106000
2       4.0    Slate  178100
3       NaN      NaN  140000


## Data Preparation

In supervised learning, we train models
to predict a designated *target* value,
given some set of *input* values. 
Our first step in processing the dataset
is to separate out columns corresponding
to input versus target values. 
We can select columns either by name or
via integer-location based indexing (`iloc`).

You might have noticed that `pandas` replaced
all CSV entries with value `NA`
with a special `NaN` (*not a number*) value. 
This can also happen whenever an entry is empty,
e.g., "3,,,270000".
These are called *missing values* 
and they are the "bed bugs" of data science,
a persistent menace that you will confront
throughout your career. 
Depending upon the context, 
missing values might be handled
either via *imputation* or *deletion*.
Imputation replaces missing values 
with estimates of their values
while deletion simply discards 
either those rows or those columns
that contain missing values. 

Here are some common imputation heuristics.
[**For categorical input fields, 
we can treat `NaN` as a category.**]
Since the `RoofType` column takes values `Slate` and `NaN`,
`pandas` can convert this column 
into two columns `RoofType_Slate` and `RoofType_nan`.
A row whose roof type is `Slate` will set values 
of `RoofType_Slate` and `RoofType_nan` to 1 and 0, respectively.
The converse holds for a row with a missing `RoofType` value.


In [3]:
inputs, targets = data.iloc[:, 0:2], data.iloc[:, 2]
print(inputs)
inputs = pd.get_dummies(inputs, dummy_na=True)
print(inputs)

   NumRooms RoofType
0       NaN      NaN
1       2.0      NaN
2       4.0    Slate
3       NaN      NaN
   NumRooms  RoofType_Slate  RoofType_nan
0       NaN           False          True
1       2.0           False          True
2       4.0            True         False
3       NaN           False          True


For missing numerical values, 
one common heuristic is to 
[**replace the `NaN` entries with 
the mean value of the corresponding column**].


In [4]:
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms  RoofType_Slate  RoofType_nan
0       3.0           False          True
1       2.0           False          True
2       4.0            True         False
3       3.0           False          True


## Conversion to the Tensor Format

Now that [**all the entries in `inputs` and `targets` are numerical,
we can load them into a tensor**] (recall :numref:`sec_ndarray`).


In [5]:
import torch

X = torch.tensor(inputs.to_numpy(dtype=float))
y = torch.tensor(targets.to_numpy(dtype=float))
X, y

(tensor([[3., 0., 1.],
         [2., 0., 1.],
         [4., 1., 0.],
         [3., 0., 1.]], dtype=torch.float64),
 tensor([127500., 106000., 178100., 140000.], dtype=torch.float64))

In [6]:
X[0], y[0]

(tensor([3., 0., 1.], dtype=torch.float64),
 tensor(127500., dtype=torch.float64))

## Discussion

You now know how to partition data columns, 
impute missing variables, 
and load `pandas` data into tensors. 
In :numref:`sec_kaggle_house`, you will
pick up some more data processing skills. 
While this crash course kept things simple,
data processing can get hairy.
For example, rather than arriving in a single CSV file,
our dataset might be spread across multiple files
extracted from a relational database.
For instance, in an e-commerce application,
customer addresses might live in one table
and purchase data in another.
Moreover, practitioners face myriad data types
beyond categorical and numeric, for example,
text strings, images,
audio data, and point clouds. 
Oftentimes, advanced tools and efficient algorithms 
are required in order to prevent data processing from becoming
the biggest bottleneck in the machine learning pipeline. 
These problems will arise when we get to 
computer vision and natural language processing. 
Finally, we must pay attention to data quality.
Real-world datasets are often plagued 
by outliers, faulty measurements from sensors, and recording errors, 
which must be addressed before 
feeding the data into any model. 
Data visualization tools such as [seaborn](https://seaborn.pydata.org/), 
[Bokeh](https://docs.bokeh.org/), or [matplotlib](https://matplotlib.org/)
can help you to manually inspect the data 
and develop intuitions about 
the type of problems you may need to address.


## Exercises

1. Try loading datasets, e.g., Abalone from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) and inspect their properties. What fraction of them has missing values? What fraction of the variables is numerical, categorical, or text?
1. Try indexing and selecting data columns by name rather than by column number. The pandas documentation on [indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) has further details on how to do this.
1. How large a dataset do you think you could load this way? What might be the limitations? Hint: consider the time to read the data, representation, processing, and memory footprint. Try this out on your laptop. What happens if you try it out on a server? 
1. How would you deal with data that has a very large number of categories? What if the category labels are all unique? Should you include the latter?
1. What alternatives to pandas can you think of? How about [loading NumPy tensors from a file](https://numpy.org/doc/stable/reference/generated/numpy.load.html)? Check out [Pillow](https://python-pillow.org/), the Python Imaging Library.


[Discussions](https://discuss.d2l.ai/t/29)


# Exercises Solution

In [7]:
data_file = os.path.join('..', 'data', 'abalone.data')
names_file = os.path.join('..', 'data', 'abalone.names')

In [27]:
column_names = ['Sex', 'Length', 'Diameter', 'Height', 'Whole_weight', 'Shucked_weight', 'Viscera_weight','Shell_weight', 'Rings']
data = pd.read_csv(data_file, header=None, names=column_names)

In [28]:
data

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.1500,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.0700,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.2100,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.1550,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.0550,7
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [29]:
data.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


In [32]:
missing_values_count = data.isna().sum() # Missing values
missing_values_count

Sex               0
Length            0
Diameter          0
Height            0
Whole_weight      0
Shucked_weight    0
Viscera_weight    0
Shell_weight      0
Rings             0
dtype: int64

In [37]:
dtypes = data.dtypes

In [38]:
data['Sex'] = data['Sex'].astype('category')

In [39]:
dtypes

Sex               category
Length             float64
Diameter           float64
Height             float64
Whole_weight       float64
Shucked_weight     float64
Viscera_weight     float64
Shell_weight       float64
Rings                int64
dtype: object

In [40]:
# Classify columns
numerical_cols = dtypes[dtypes.apply(lambda x: x.kind in 'biufc')]  # integer, unsigned integer, float, complex numbers
categorical_cols = dtypes[dtypes.apply(lambda x: x.name == 'category' or x.name == 'object')]  # categorical or object (string)
text_cols = dtypes[dtypes == 'object']  # object type specifically for text

In [41]:
# Calculate fractions
total_columns = len(data.columns)
fraction_numerical = len(numerical_cols) / total_columns
fraction_categorical = len(categorical_cols) / total_columns
fraction_text = len(text_cols) / total_columns

In [42]:
# Output the fractions
print(f"Fraction of numerical variables: {fraction_numerical:.2f}")
print(f"Fraction of categorical variables: {fraction_categorical:.2f}")
print(f"Fraction of text variables: {fraction_text:.2f}")

Fraction of numerical variables: 0.89
Fraction of categorical variables: 0.11
Fraction of text variables: 0.00


In [47]:
print(data['Sex'][13])

F


In [48]:
data.describe(include = "all")

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,Rings
count,4177,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000,4177.000000
unique,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,M,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,0.523992,0.407881,0.139516,0.828742,0.359367,0.180594,0.238831,9.933684
std,NaN,0.120093,0.099240,0.041827,0.490389,0.221963,0.109614,0.139203,3.224169
min,NaN,0.075000,0.055000,0.000000,0.002000,0.001000,0.000500,0.001500,1.000000
25%,NaN,0.450000,0.350000,0.115000,0.441500,0.186000,0.093500,0.130000,8.000000
50%,NaN,0.545000,0.425000,0.140000,0.799500,0.336000,0.171000,0.234000,9.000000
75%,NaN,0.615000,0.480000,0.165000,1.153000,0.502000,0.253000,0.329000,11.000000


In [49]:
data[["Sex", "Rings", "Length"]][ : 20]

,Sex,Rings,Length
0,M,15,0.455
1,M,7,0.350
2,F,9,0.530
3,M,10,0.440
4,I,7,0.330
5,I,8,0.425
6,F,20,0.530
7,F,16,0.545
8,M,9,0.475
9,F,19,0.550
